In [181]:
import pandas as pd
import numpy as np
from PIL import Image,ImageFilter
import matplotlib.pyplot as plt

In [225]:
# This function will get the pixels of each partition

def get_pixels(num_pixels_x, num_tiles_x):
    tile_pixels_x = np.zeros(3)
    if num_pixels_x % num_tiles_x == 0:
        tile_pixels_x = num_pixels_x/num_tiles_x
    else:
        tile_pixels_x[2] = int(num_pixels_x/num_tiles_x)
        remain = num_pixels_x % num_tiles_x
        if remain % 2 == 0:
            tile_pixels_x[0] = tile_pixels_x[1] + remain/2
            tile_pixels_x[2] = tile_pixels_x[1] + remain/2
        else:
            tile_pixels_x[0] = tile_pixels_x[1] + int(remain/2)
            tile_pixels_x[2] = tile_pixels_x[1] + int(remain/2) + 1

    x1 = np.zeros(num_tiles_x + 1)
    x1[1] = tile_pixels_x[0]
    x1[2:-1] = tile_pixels_x[1]
    x1[-1] = tile_pixels_x[2]

    return x1


# This method will randomly allocate the extra pixels to tiles
# return the 刻度
def get_pixels_ran(num_pixels_x, num_tiles_x):
    tile_pixels_x = np.zeros(num_tiles_x)
    if num_pixels_x % num_tiles_x == 0:
        tile_pixels_x = (num_pixels_x/num_tiles_x)*np.ones(num_tiles_x).astype(int)
    else:
        tile_pixels_x = int(num_pixels_x/num_tiles_x)*np.ones(num_tiles_x).astype(int)
        remain = num_pixels_x % num_tiles_x
        position = np.random.randint(0,num_tiles_x, size = remain)

        for i in range(remain):
            tile_pixels_x[position[i]] += 1

    cum_sum = np.cumsum(tile_pixels_x).astype(int)
    tile_pixels_x = cum_sum - int(tile_pixels_x[0])
    tile_pixels_x = np.append(tile_pixels_x,num_pixels_x)
    return tile_pixels_x
        
        


# This function will calculate the chi_square
def chi_square(X,Y,bins):
    # First Get the histogram with given bins of
    # Cost too much time 
    hist_x = np.zeros((3,bins))
    hist_y = np.zeros((3,bins))
    d = np.zeros((3,1))

    for i in range(3):
        hist_x[i], cnt = np.histogram(X[:,:,i], bins = bins) #ax[i,0].hist(X[:,:,i])
        hist_x[i] = hist_x[i]/np.sum(hist_x[i])
        hist_y[i], cnt = np.histogram(Y[:,:,i], bins = bins)
        hist_y[i] = hist_y[i]/np.sum(hist_y[i])
        d[i] = np.sum(np.square(hist_x[i] - hist_y[i])/(hist_x[i] + hist_y[i]+1))

    return np.sqrt(np.sum(np.square(d)))

# I is the original image, x and y is the output image
def gauss_subsample(I,x,y):
    I = I.filter(ImageFilter.GaussianBlur)
    I = I.resize((x,y))
    return I 

    





In [226]:
# test chi
# I1 = Image.open('data/target/target1.jpg') 
# I2 = Image.open('data/target/target2.jpg')
# I1 = np.array(I1)
# I2 = np.array(I2)
# chi_square(I1,I1,20)

# test random 
get_pixels_ran(1698,3)


[566. 566. 566.]


array([   0,  566, 1132, 1698])

In [186]:
# 首先看图片大小是否一样,不是很好存
with open('data/dataset/manmade_training.txt','r',encoding='utf-8') as f:
    im_name = [line.strip('\n') for line in f.readlines()]

size = np.zeros((len(im_name),2))  
cnt = 0
for path in im_name:
    path = 'data/dataset/' + path
    im = np.array(Image.open(path))
    size[cnt] = np.shape(im[:,:,0])
    cnt += 1

size



array([[1024.,  768.],
       [1290., 2032.],
       [1244.,  877.],
       [ 851., 1280.],
       [2200., 1650.],
       [ 240.,  320.],
       [ 400.,  600.],
       [ 224.,  400.],
       [ 600.,  800.],
       [1024., 1536.],
       [ 330.,  250.],
       [2304., 3072.],
       [ 480.,  640.],
       [ 545.,  741.],
       [ 275.,  204.],
       [2448., 3264.],
       [1704., 2272.],
       [ 316.,  200.],
       [ 960., 1280.],
       [ 600.,  800.],
       [ 256.,  256.],
       [ 768., 1024.],
       [ 384.,  256.],
       [ 202.,  310.],
       [2112., 2816.],
       [1577., 2221.],
       [ 300.,  224.],
       [ 256.,  256.],
       [1536., 2048.],
       [ 256.,  256.],
       [ 480.,  640.],
       [ 402.,  600.],
       [ 240.,  320.],
       [2304., 3456.],
       [ 800.,  481.],
       [ 256.,  256.],
       [2592., 1944.],
       [1368., 1712.],
       [ 549.,  732.],
       [1320.,  960.],
       [ 240.,  320.],
       [ 240.,  320.],
       [2448., 3264.],
       [ 96

In [227]:
import heapq


def main(target,pixels_x, pixels_y, tiles_x,tiles_y):
    # first get the partition of original pic
    # x1,y1 表示原始图像中的分割
    # x2,y2 表示输出图像中的分割大小，找到对应图片后应该压缩到的大小
    x1 = get_pixels_ran(np.shape(target[:,:,0])[1], tiles_x).astype(int)
    y1 = get_pixels_ran(np.shape(target[:,:,0])[0], tiles_y).astype(int)

    # construct x and y with the size of tiles
    x2 = get_pixels_ran(pixels_x, tiles_x).astype(int)
    y2 = get_pixels_ran(pixels_y, tiles_y) .astype(int)

    print(x2,y2)


    # result image
    result = np.zeros((pixels_y, pixels_x, 3))


    # 将所有图片读成两个数组
    # TODO
    with open('data/dataset/manmade_training.txt','r',encoding='utf-8') as f:
        im_name_m = [line.strip('\n') for line in f.readlines()]
    with open('data/dataset/natural_training.txt','r',encoding='utf-8') as f:
        im_name_n = [line.strip('\n') for line in f.readlines()]

    # Method 2: Split images into diff parts 
    dict_images_m = {}
    dict_images_n = {}
    for path in im_name_m:
        path = 'data/dataset/' + path
        im = Image.open(path)
        dict_images_m[path] = np.array(im)

               
            
    for path in im_name_n:
        path = 'data/dataset/' + path
        im = Image.open(path)
        dict_images_n[path] = np.array(im)
    

    for i in range(tiles_y):
        for j in range(tiles_x):
            # if (i == tiles_y -1 )&(j != tiles_x -1):
            #     target_ij = target[y1[i]:y1[i+1] + 1,x1[j]:x1[j+1]]
            #     y_pixels = y2[i+1] + 1 - y2[i]
            #     x_pixels = x2[j+1] - x2[j]

            # elif (i != tiles_y -1) & (j == tiles_x -1):
            #     target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1] + 1]
            #     y_pixels = y2[i+1] - y2[i]
            #     x_pixels = x2[j+1] + 1 - x2[j]

            # elif (i == tiles_y -1) & (j == tiles_x -1):
            #     target_ij = target[y1[i]:y1[i+1] + 1,x1[j]:x1[j+1] + 1]
            #     y_pixels = y2[i+1] + 1 - y2[i]
            #     x_pixels = x2[j+1] + 1 - x2[j]

            # else:
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []

            # Method 1: read one image then get a distance between the target and each image 
            # for path in im_name_m:
            #     path = 'data/dataset/' + path
            #     im = Image.open(path)
                
            #     d = chi_square(target_ij,np.array(im),20)
            #     dict_path[path] = d
            
            # for path in im_name_n:
            #     path = 'data/dataset/' + path
            #     im = Image.open(path)
                
            #     d = chi_square(target_ij,np.array(im),20)
            #     dict_path[path] = d

            for item in dict_images_m.items():
                dict_path[item[0]] = chi_square(target_ij,item[1],20)

            for item in dict_images_n.items():
                dict_path[item[0]] = chi_square(target_ij,item[1],20)

            top = heapq.nsmallest(1,dict_path)
            print("Get the patch of %s, %s" % (i,j))
            print(top)

            # Then fit the image into the proper size: Using Gaussian subsample
            # First get the Gaussian smoothing
            # Then get the subsample with the proper size
            im = Image.open(top[0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)
            print(x_pixels,y_pixels)

            # 向结果矩阵中拼接
            # if (i == tiles_y -1) & (j != tiles_x - 1):
            #     result[y2[i]: , x2[j]: x2[j+1]] = np.array(im)
            # elif (i != tiles_y - 1) & (j == tiles_x - 1):
            #     result[y2[i]: y2[i+1], x2[j]: ] = np.array(im)
            # elif (i == tiles_y - 1) & (j == tiles_x - 1):
            #     result[y2[i]:  , x2[j]: ] = np.array(im)
            # else: 
            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)

                
            # two way to impelement : 


            
    return result



            

In [228]:
result = main(np.array(Image.open('data/target/target1.jpg')), 101,101,3,3)
result.show()

[566. 566. 566.]
[377. 377. 377.]
[34 34 33]
[33 35 33]
[  0  34  67 101] [  0  35  68 101]
Get the patch of 0, 0
['data/dataset/manmade_training/sun_aabghtsyctpcjvlc.jpg']
34 35
Get the patch of 0, 1
['data/dataset/manmade_training/sun_aabghtsyctpcjvlc.jpg']
33 35
Get the patch of 0, 2
['data/dataset/manmade_training/sun_aabghtsyctpcjvlc.jpg']
34 35
Get the patch of 1, 0
['data/dataset/manmade_training/sun_aabghtsyctpcjvlc.jpg']
34 33
Get the patch of 1, 1
['data/dataset/manmade_training/sun_aabghtsyctpcjvlc.jpg']
33 33
Get the patch of 1, 2
['data/dataset/manmade_training/sun_aabghtsyctpcjvlc.jpg']
34 33
Get the patch of 2, 0
['data/dataset/manmade_training/sun_aabghtsyctpcjvlc.jpg']
34 33
Get the patch of 2, 1
['data/dataset/manmade_training/sun_aabghtsyctpcjvlc.jpg']
33 33
Get the patch of 2, 2
['data/dataset/manmade_training/sun_aabghtsyctpcjvlc.jpg']
34 33


AttributeError: 'numpy.ndarray' object has no attribute 'show'

In [240]:
Image.fromarray(np.uint8(result)).save('results/para(101,1).jpg')

In [146]:
I = np.array(Image.open('data/target/target1.jpg'))
Image.fromarray(I[0:4,0:4])


In [158]:
dict_test = {}
dict_test['a'] = 1


In [169]:
dict_test.items()

dict_items([('a', 1)])

In [192]:
for item in dict_test.items():
    print(item[0])

a
